In [1]:
# cd.. In separate cell if used

from lib.network_mapper import network_mapper_function
from lib.NR3 import NR3_function
from lib.create_linear_constraints_base import create_linear_constraints_base_function
from lib.parse_CVX_output import parse_CVX_output_function
#from lib.computeFTreal import computeFTrealfunction
#from lib.computeJTreal import computeJTrealfunction
#from lib import *
import numpy as np
from cvxopt import matrix
from cvxopt import solvers
import math
import matplotlib.pyplot as plt
#matplotlib inline
fig_size = np.array([18, 6])
plt.rcParams["figure.figsize"] = fig_size

# z = exp(s*T)

# zoh
# z ~ 1 + s*T
# s ~ (z - 1)/T

# bilinear transform
# z ~ (2 + sT)/(2 - sT)
# s ~ (2/T)(z - 1)/(z + 1)

In [2]:
fp = '/home/michael/Dropbox/Unbalanced LinDistflow/20180101/NETWORKS/'
fn = 'ieee_13node_balance.txt'
fn = '05node_fullphase_radial.txt'

network1 = network_mapper_function(fp, fn)

nnode1 = network1.nodes.nnode
nline1 = network1.lines.nline

network1.loads.aPQ = 1.00*np.ones((3,nnode1))*(network1.loads.spu != 0)
network1.loads.aI = 0.00*np.ones((3,nnode1))*(network1.loads.spu != 0)
network1.loads.aZ = 0.00*np.ones((3,nnode1))*(network1.loads.spu != 0)

BASE
Vbase: 5000.0
Sbase: 5000000.0
Ibase: 1000.0
Zbase: 5.0

NODES
nnode: 6
nodelist: ['A0', 'A1', 'A2', 'A3', 'A4', 'A5']
phases: ['abc', 'abc', 'abc', 'abc', 'abc', 'abc']
PH:
 [[1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]]

LINES
nline: 5
TXnode: ['A0', 'A1', 'A2', 'A2', 'A4']
RXnode: ['A1', 'A2', 'A3', 'A4', 'A5']
TXnum: [0 1 2 2 4]
RXnum: [1 2 3 4 5]
phases: ['abc', 'abc', 'abc', 'abc', 'abc']
PH:
 [[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
config: ['12', '12', '12', '12', '12']
length: [ 76.2 304.8 228.6 228.6 228.6]

NODES + LINES
inlines:
 [[-1  0  1  2  3  4]]
innodes:
 [[-1  0  1  2  2  4]]
outlines:
 [[ 0  1  2 -1  4 -1]
 [-1 -1  3 -1 -1 -1]]
outnodes:
 [[ 1  2  3 -1  5 -1]
 [-1 -1  4 -1 -1 -1]]

CONFIGS
nconf: 3
conflist: ['12', '13', '14']
12 - FZpl:
 [[4.30611306e-05+1.26499062e-04j 1.93868294e-05+6.23485404e-05j
  1.96353785e-05+5.26426982e-05j]
 [1.93868294e-05+6.23485404e-05j 4.19426597e-05+1.30214871e-04j
  1.90761430e-05+4.78332733e-05j]
 [1.96353785e-05+5.26426982e-05j

In [3]:
#####-----#####-----#####

class simclass():
    pass
sim = simclass()

sim.slacknode = 0
sim.Vslack = np.matrix([[1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)]]).T

# node parameters
nnode = network1.nodes.nnode;

# line parameters
nline = network1.lines.nline;
TXnum = network1.lines.TXnum;
RXnum = network1.lines.RXnum;
FZpu = network1.lines.FZpu;

# load parameters
spu = network1.loads.spu;
aPQ = network1.loads.aPQ;
aI = network1.loads.aI;
aZ = network1.loads.aZ;

# capacitor paramters
cappu = network1.caps.cappu;

# controller parameters
wmaxpu = network1.cons.wmaxpu;

# sim parameters
slacknode = sim.slacknode;
Vslack = sim.Vslack

# iteration paramaters
if hasattr(sim, 'Vkm1'):
    Vkm1 = sim.Vkm1
else:
    Vkm1 = np.ones((3,nnode), dtype='complex')
    Vkm1[1,:] = np.exp(1j*-120*np.pi/180)
    Vkm1[2,:] = np.exp(1j*120*np.pi/180)

if hasattr(sim, 'Lkm1'):
    Lkm1 = sim.Lkm1
else:
    Lkm1 = np.zeros((3,nline))
    
if hasattr(sim, 'Hkm1'):
    Hkm1 = sim.Hkm1
else:
    Hkm1 = np.zeros((3,nline))


In [4]:
nvar, Aineq, bineq, Aeq, beq = create_linear_constraints_base_function(network1,sim,sim.slacknode,sim.Vslack)

tn1 = 3
tn2 = 5

P = np.zeros((3*nvar,3*nvar))
for ph in range(0,3):
    P[ph*nvar+tn1,ph*nvar+tn1] = 1
    P[ph*nvar+tn2,ph*nvar+tn2] = 1
    P[ph*nvar+tn1,ph*nvar+tn2] = -1
    P[ph*nvar+tn2,ph*nvar+tn1] = -1
Pcvx = matrix(P, tc='d')
    
Q = np.zeros((3*nvar,1))
Qcvx = matrix(Q, tc='d')

Aineqcvx = matrix(Aineq, tc='d')
bineqcvx = matrix(bineq, tc='d')
Aeqcvx = matrix(Aeq, tc='d')
beqcvx = matrix(beq, tc='d')

sol = solvers.qp(Pcvx,Qcvx,Aineqcvx,bineqcvx,Aeqcvx,beqcvx)
print(sol['status'])
X = sol['x']

XX = np.zeros((3*nvar,1))
for k1 in range(0,3*nvar):
    XX[k1] = X[k1]
XX[np.abs(XX) <= 1e-9] = 0

# print(XX)

Eopt, Topt, Vopt, Iopt, Sopt, wopt, dopt, sopt = parse_CVX_output_function(X,nvar,network1)

print('Eopt\n', Eopt)
print('Topt\n', Topt)
print('Vopt\n', Vopt)
print('Iopt\n', Iopt)
print('Sopt\n', Sopt)
print('wopt\n', wopt)
print('dopt\n', dopt)
print('sopt\n', sopt)


     pcost       dcost       gap    pres   dres
 0:  1.3209e-04 -2.2484e+02  2e+02  5e-16  3e+00
 1:  1.3208e-04 -2.2496e+00  2e+00  1e-16  3e-02
 2:  1.3160e-04 -2.3611e-02  2e-02  2e-16  3e-04
 3:  9.5633e-05 -1.0989e-03  1e-03  1e-16  1e-05
 4:  4.4160e-06 -9.3931e-05  1e-04  1e-16  4e-07
 5:  6.0852e-09 -2.4408e-06  2e-06  1e-16  3e-09
 6:  6.2935e-13 -2.4667e-08  2e-08  1e-16  3e-11
Optimal solution found.
optimal
Eopt
 [[1.       1.00188  1.009398 1.010299 1.010134 1.010298]
 [1.       0.999881 0.999407 0.999259 1.000116 0.999259]
 [1.       1.003249 1.016247 1.019726 1.019372 1.019726]]
Topt
 [[   0.          0.198382    0.99191     1.180008    1.09859     1.169515]
 [-120.       -119.879403 -119.397013 -119.326659 -119.357185 -119.294965]
 [ 120.        120.042135  120.30539   120.161719  120.511757  120.367939]]
Vopt
 [[ 1.      +0.      j  1.000934+0.003466j  1.004537+0.017392j
   1.004923+0.020699j  1.004869+0.01927 j  1.004926+0.020515j]
 [-0.5     -0.866025j -0.498146-0.86

In [5]:
network1.cons.wpu = wopt

slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])

VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
    NR3_function(network1,sim.slacknode,sim.Vslack,[],[],100)
    
VNR = np.round(VNR,decimals=6)
INR = np.round(INR,decimals=6)
STXNR = np.round(STXNR,decimals=6)
SRXNR = np.round(SRXNR,decimals=6)
iNR = np.round(iNR,decimals=6)
sNR = np.round(sNR,decimals=6)

'''
VNR = np.floor(1e6*VNR)/1e6
INR = np.floor(1e6*INR)/1e6
STXNR = np.floor(1e6*STXNR)/1e6
SRXNR = np.floor(1e6*SRXNR)/1e6
iNR = np.floor(1e6*iNR)/1e6
sNR = np.floor(1e6*sNR)/1e6
'''

print('itercount:', itercount)
print('VNR\n', VNR)
print('INR\n', INR)
print('STXNR\n', STXNR)
print('SRXNR\n', SRXNR)
print('iNR\n', iNR)
print('sNR\n', sNR)

itercount: 4
VNR
 [[ 1.      +0.      j  0.99637 -0.002756j  0.98185 -0.013782j
   0.979272-0.016131j  0.980271-0.01527 j  0.979373-0.016233j]
 [-0.5     -0.866025j -0.501005-0.864236j -0.505023-0.857077j
  -0.505662-0.856058j -0.505071-0.856788j -0.505896-0.855978j]
 [-0.5     +0.866025j -0.496684+0.863602j -0.483418+0.853907j
  -0.483853+0.852235j -0.477314+0.851706j -0.477761+0.851617j]]
INR
 [[ 0.544876-0.415977j  0.544876-0.415977j  0.133303-0.061233j
   0.088866-0.126148j  0.061247-0.001015j]
 [-0.439492-0.178513j -0.439492-0.178513j -0.105122-0.012306j
  -0.056003-0.05021 j -0.045791-0.032887j]
 [ 0.074021+0.617542j  0.074021+0.617542j  0.041562+0.033324j
  -0.006678+0.281   j  0.      +0.      j]]
STXNR
 [[0.544876+4.15977e-01j 0.544045+4.12965e-01j 0.131728+5.82840e-02j
  0.088992+1.22633e-01j 0.060054+6.00000e-05j 0.      +0.00000e+00j]
 [0.374343+2.91354e-01j 0.374465+2.90388e-01j 0.063636+8.38830e-02j
  0.071317+2.26420e-02j 0.051305+2.26230e-02j 0.      +0.00000e+00j]
 [0.

In [6]:
print(network1.loads.spu)

[[0.  +0.  j 0.  +0.  j 0.32+0.22j 0.1 +0.1 j 0.06+0.08j 0.06+0.04j]
 [0.  +0.  j 0.  +0.  j 0.24+0.18j 0.1 +0.1 j 0.02+0.  j 0.  +0.  j]
 [0.  +0.  j 0.  +0.  j 0.24+0.18j 0.01+0.  j 0.24+0.18j 0.  +0.  j]]
